In [ ]:
import json
summaries = []
with open('/content/drive/MyDrive/articles/summaries1.json', 'r') as f:
    summaries = json.load(f)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
summaries[0]

{'date': '2002-03-01T00:00:00+05:30',
 'company': 'INFY',
 'content': 'Another budget has come and gone badly, and this one too has been hounded by bad luck. The Sensex has dropped 175 points intraday . But once again, finance minister Yashwant Sinha seems to have deliberately attacked investor sentiment by not cutting the dividend tax. Instead, he has given a dividend tax, a surcharge on income tax, and a corporate tax. Overall, though, the budget has many good features, such as reforms to agriculture, power, urban renewal, education, and infrastructure, which were largely ignored by the market. The disappointment over the budget is also a reflection of high expectations that were built in the run up to it," the Bank of England\'s deputy chief investment officer R Balakrishnan writes. He argues that the reduction of the savings rate by 50 basis points not only hurt small savers and pensioners but it also disappointed industry. Similarly, certain of Sinha\'s proposals, like allowing mu

In [ ]:
import json
from tqdm import tqdm

import json
summaries = []
with open('/content/drive/MyDrive/articles/summaries1.json', 'r') as f:
    summaries.extend(json.load(f))
with open('/content/drive/MyDrive/articles/summaries2.json', 'r') as f:
    summaries.extend(json.load(f))
with open('/content/drive/MyDrive/articles/summaries3.json', 'r') as f:
    summaries.extend(json.load(f))
with open('/content/drive/MyDrive/articles/summaries4.json', 'r') as f:
    summaries.extend(json.load(f))
with open('/content/drive/MyDrive/articles/summaries5.json', 'r') as f:
    summaries.extend(json.load(f))
with open('/content/drive/MyDrive/articles/summaries6.json', 'r') as f:
    summaries.extend(json.load(f))

unique_articles = []
def count_unique_articles(summaries):
    for summary in tqdm(summaries, desc="Processing summaries"):
        is_unique = True
        for unique_summary in unique_articles:
            if summary['content'] == unique_summary['content']:
                is_unique = False
                break
        if is_unique:
            unique_articles.append(summary)
    return len(unique_articles)

unique_article_count = count_unique_articles(summaries)
print(f"\n\nNumber of unique articles: {unique_article_count}")

Processing summaries: 100%|██████████| 99230/99230 [17:23<00:00, 95.09it/s]



Number of unique articles: 60600


In [ ]:
import json
unique_articles = []
with open('/content/drive/MyDrive/articles/unique_summaries.json', 'r') as f:
    unique_articles = json.load(f)

In [ ]:
!pip uninstall -y tensorflow sentence-transformers huggingface_hub tokenizers
!pip install tensorflow sentence-transformers huggingface_hub tokenizers

Found existing installation: sentence-transformers 3.2.1
Uninstalling sentence-transformers-3.2.1:
  Successfully uninstalled sentence-transformers-3.2.1
Found existing installation: huggingface-hub 0.26.2
Uninstalling huggingface-hub-0.26.2:
  Successfully uninstalled huggingface-hub-0.26.2
Found existing installation: tokenizers 0.20.3
Uninstalling tokenizers-0.20.3:
  Successfully uninstalled tokenizers-0.20.3
  Using cached tensorboard-2.18.0-py3-none-any.whl.metadata (1.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 447.6/447.6 kB 28.3 MB/s eta 0:00:00
Using cached tensorboard-2.18.0-py3-none-any.whl (5.5 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 61.7 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
      Succes

In [ ]:
from sentence_transformers import SentenceTransformer

# Load the high-accuracy model
model = SentenceTransformer('all-mpnet-base-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from tqdm import tqdm

# Extract the content from each article
article_contents = [article['content'] for article in unique_articles]

# Encode the articles in batches with a progress bar
batch_size = 128  # Adjust the batch size based on your hardware capability
embeddings = []

for i in tqdm(range(0, len(article_contents), batch_size), desc="Encoding Articles"):
    batch = article_contents[i:i + batch_size]
    batch_embeddings = model.encode(batch)
    embeddings.extend(batch_embeddings)

# Add the embeddings back to the unique_articles list
for i, emb in enumerate(embeddings):
    unique_articles[i]['embedding'] = emb.tolist()

Encoding Articles: 100%|██████████| 474/474 [20:42<00:00,  2.62s/it]


In [ ]:
import json
embeddings = []
with open('/content/drive/MyDrive/articles/unique_embeddings.json', 'r') as f:
    embeddings = json.load(f)

In [ ]:
from datetime import datetime
for article in embeddings:
    article['date'] = datetime.fromisoformat(article['date'])

# Sort the list based on the date
embeddings.sort(key=lambda x: x['date'])

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

past_embeddings = np.array([])
min_similarity_threshold = 0.8

past_similarities = []
# Iterate over each article
for idx, current_article in enumerate(tqdm(embeddings, desc="Calculating similarities")):
    current_company = current_article['company']
    current_embedding = np.array(current_article['embedding']).reshape(1, -1)

    if past_embeddings.size == 0:
        past_embeddings = current_embedding
        continue

    # Compute cosine similarities
    similarities = cosine_similarity(current_embedding, past_embeddings)

    # Find the index of the most similar article
    most_similar_index = np.argmax(similarities)
    most_similar_score = similarities[0, most_similar_index]

    if most_similar_score >= min_similarity_threshold:
        similarity = {
            'current_date': current_article['date'],
            'current_company': current_company,
            'most_similar_date': embeddings[most_similar_index]['date'],
            'most_similar_company': embeddings[most_similar_index]['company'],
            'most_similar_index': most_similar_index,
            'similarity_score': most_similar_score,
            'differece_vector': (current_embedding - past_embeddings[most_similar_index]).tolist()
        }
        past_similarities.append(similarity)
    past_embeddings = np.vstack((past_embeddings, current_embedding))

with open('/content/drive/MyDrive/articles/similarities.json', 'w') as f:
    json.dump(past_similarities, f)

Calculating similarities: 100%|██████████| 60600/60600 [4:47:52<00:00,  3.51it/s]


TypeError: Object of type ndarray is not JSON serializable

In [ ]:
for similarity in past_similarities:
    similarity['current_date'] = similarity['current_date'].strftime('%Y-%m-%d')
    similarity['most_similar_date'] = similarity['most_similar_date'].strftime('%Y-%m-%d')

In [ ]:
for similarity in past_similarities:
    similarity['most_similar_index'] = int(similarity['most_similar_index'])

In [ ]:
for similarity in past_similarities:
    similarity['similarity_score'] = float(similarity['similarity_score'])

In [ ]:
type(past_similarities[0]['similarity_score'])

float

In [ ]:
with open('/content/drive/MyDrive/articles/similarities.json', 'w') as f:
    json.dump(past_similarities, f)